<a href="https://colab.research.google.com/github/Judy-Choi/Tensorflow_Certificate/blob/main/13_sunspot_lamda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Val_loss = 13.3
val_acc = 13.7369

요즈음에는 lambda 버전은 잘 안나옴..

In [ ]:
!pip uninstall tensorflow
!pip install tensorflow==2.10.0

# 답안

In [ ]:
# ======================================================================
# There are 5 questions in this test with increasing difficulty from 1-5
# Please note that the weight of the grade for the question is relative
# to its difficulty. So your Category 1 question will score much less
# than your Category 5 question.
# ======================================================================
#
# Sequence Modelling Question.
#
# For this task you will need to train a neural network
# to predict sunspot activity using the Sunspots.csv
# provided. Your neural network is expected to have an MAE
# of at least 20, with top marks going to one with an MAE
# of around 15. At the bottom is provided some testing
# code should you want to check before uploading which measures
# the MAE for you. Strongly recommend you test your model with
# this to be able to see how it performs.
import csv
import tensorflow as tf
import numpy as np
import urllib
from tensorflow.keras.layers import Dense, LSTM, Lambda, Conv1D,Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import Huber

# DO NOT CHANGE THIS CODE
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    # Window Sequence Loader 만들기
    series = tf.expand_dims(series, axis=-1)
    # Dataset loader 만들기
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)


def solution_model():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv'
    urllib.request.urlretrieve(url, 'sunspots.csv')

    # Your data should be loaded into 2 Python lists called time_step
    # and sunspots. They are decleared here.
    time_step = []
    sunspots = []

    with open('sunspots.csv') as csvfile:
      reader = csv.reader(csvfile, delimiter=',')
      next(reader)
      for row in reader:
        sunspots.append(float(row[2]))# YOUR CODE HERE
        time_step.append(int(row[0]))

    
    # You should use numpy to create 
    # - your series from the list of sunspots
    # - your time details from the list of time steps
    series = np.array(sunspots) # YOUR CODE HERE
    time = np.array(time_step)

    # You should split the dataset into training and validation splits
    # At time 3000. So everything up to 3000 is training, and everything
    # after 3000 is validation. Write the code below to achieve that.
    split_time = 3000

    # YOUR CODE HERE
    time_train = time[:split_time]
    x_train = series[:split_time]
    time_valid = time[split_time:]
    x_valid = series[split_time:]
    # DO NOT CHANGE THIS CODE
    window_size = 30
    batch_size = 32
    shuffle_buffer_size = 1000


    tf.keras.backend.clear_session()
    # You can use any random seed you want. We use 53. :)
    tf.random.set_seed(53)
    np.random.seed(53)
    train_set = windowed_dataset(x_train, window_size=window_size, batch_size=batch_size, shuffle_buffer=shuffle_buffer_size)
    
    validation_set = windowed_dataset(x_valid, 
        window_size=window_size,
        batch_size=batch_size,
        shuffle_buffer=shuffle_buffer_size)


    model = Sequential([
        # YOUR CODE HERE. Whatever your first layer is, 
        # the input shape will be [None,1] when using the Windowed_dataset above, 
        # depending on the layer type chosen
        Conv1D(filters=16, kernel_size=5,
            padding="causal",
            activation="relu",
            input_shape=[None, 1]),
        Bidirectional(LSTM(64, return_sequences=True)),
        Bidirectional(LSTM(64, return_sequences=True)),
        Dropout(0.5),
        Dense(64, activation="relu"),
        Dense(32, activation="relu"),       

    # YOUR CODE HERE. DO NOT CHANGE THE FINAL TWO LAYERS FROM BELOW
        Dense(1),
    # The data is not normalized, so this lambda layer helps
    # keep the MAE in line with expectations. Do not modify.
        Lambda(lambda x: x * 400)
    ])

   
    # YOUR CODE HERE TO COMPILE AND TRAIN THE MODEL
    optimizer = SGD(learning_rate=1e-4, momentum=0.9)
    loss= Huber()
    model.compile(loss=loss,
        optimizer=optimizer,
        metrics=["mae"])
    
    checkpoint_path = 'checkpoint.ckpt'
    checkpoint = ModelCheckpoint(checkpoint_path, 
        save_weights_only=True, 
        save_best_only=True, 
        monitor='val_mae',
        verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)
    epochs=150

    history = model.fit(train_set, 
        validation_data=(validation_set), 
        epochs=epochs, 
        callbacks=[checkpoint, early_stopping],
        )
    model.load_weights(checkpoint_path)
    return model


# Note that you'll need to save your model as a .h5 like this
# This .h5 will be uploaded to the testing infrastructure
# and a score will be returned to you
if __name__ == '__main__':
    model = solution_model()
    model.save("tf5-sunspot-3.h5")

# 시험때 다음 줄은 복사하지 마세요
    # from google.colab import files
    # files.download('tf5-sunspot-3.h5')

Epoch 1/150
     93/Unknown - 24s 15ms/step - loss: 36.8542 - mae: 37.3504
Epoch 1: val_mae improved from inf to 18.29640, saving model to checkpoint.ckpt
93/93 [==============================] - 26s 37ms/step - loss: 36.8542 - mae: 37.3504 - val_loss: 17.8041 - val_mae: 18.2964
Epoch 2/150
93/93 [==============================] - ETA: 0s - loss: 22.7031 - mae: 23.1972
Epoch 2: val_mae did not improve from 18.29640
93/93 [==============================] - 3s 25ms/step - loss: 22.7031 - mae: 23.1972 - val_loss: 18.6595 - val_mae: 19.1531
Epoch 3/150
90/93 [============================>.] - ETA: 0s - loss: 17.6839 - mae: 18.1763
Epoch 3: val_mae improved from 18.29640 to 17.19674, saving model to checkpoint.ckpt
93/93 [==============================] - 2s 16ms/step - loss: 17.7402 - mae: 18.2326 - val_loss: 16.7063 - val_mae: 17.1967
Epoch 4/150
90/93 [============================>.] - ETA: 0s - loss: 16.5308 - mae: 17.0227
Epoch 4: val_mae improved from 17.19674 to 11.40538, saving mode

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ChatGPT

Q) 다음 코드의 성능을 개선해 줘.
참고로 사용하는 데이터는 시계열 데이터야.

A) Batch normalization
You can try adding batch normalization after the first convolutional layer and after each LSTM layer. This can improve the convergence of the model and lead to better performance.

Reduce the number of LSTM layers
Having two stacked Bidirectional LSTM layers can be computationally expensive, especially for long sequences. You can try reducing the number of LSTM layers to one and increase the number of filters in the convolutional layer instead.

Increase the number of filters in the convolutional layer
Increasing the number of filters in the convolutional layer can help capture more relevant features from the input sequence. You can try increasing the number of filters from 16 to a larger number.

Add an additional dense layer
You can try adding an additional dense layer before the output layer to allow for more complex interactions between the features extracted from the input sequence.


Q) 사용하는 데이터가 float 자료형 시계열 데이터임을 감안해서 다시 코드를 개선해 줘

A) 각 레이어의 유닛 수와 하이퍼파라미터는 데이터에 따라 최적화되어야 하므로 이전에 제안한 개선 사항은 유일한 해결책이 아닙니다. 하지만 데이터가 float 자료형 시계열 데이터인 경우, 이전에 제안한 모델을 사용하되 float32 자료형을 사용하도록 수정할 수 있습니다. 이렇게 하면 모델의 속도를 높일 수 있습니다.

모든 레이어에서 dtype 매개 변수를 "float32"로 설정하고, 모델 컴파일 단계에서 "float32"로 데이터 유형을 지정할 수 있습니다.

In [ ]:
# ======================================================================
# There are 5 questions in this test with increasing difficulty from 1-5
# Please note that the weight of the grade for the question is relative
# to its difficulty. So your Category 1 question will score much less
# than your Category 5 question.
# ======================================================================
#
# Sequence Modelling Question.
#
# For this task you will need to train a neural network
# to predict sunspot activity using the Sunspots.csv
# provided. Your neural network is expected to have an MAE
# of at least 20, with top marks going to one with an MAE
# of around 15. At the bottom is provided some testing
# code should you want to check before uploading which measures
# the MAE for you. Strongly recommend you test your model with
# this to be able to see how it performs.
import csv
import tensorflow as tf
import numpy as np
import urllib
from tensorflow.keras.layers import Dense, LSTM, Lambda, Conv1D,Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber

# DO NOT CHANGE THIS CODE
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    # Window Sequence Loader 만들기
    series = tf.expand_dims(series, axis=-1)
    # Dataset loader 만들기
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)


def solution_model():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv'
    urllib.request.urlretrieve(url, 'sunspots.csv')

    # Your data should be loaded into 2 Python lists called time_step
    # and sunspots. They are decleared here.
    time_step = []
    sunspots = []

    with open('sunspots.csv') as csvfile:
      reader = csv.reader(csvfile, delimiter=',')
      next(reader)
      for row in reader:
        sunspots.append(float(row[2]))# YOUR CODE HERE
        time_step.append(int(row[0]))

    
    # You should use numpy to create 
    # - your series from the list of sunspots
    # - your time details from the list of time steps
    series = np.array(sunspots) # YOUR CODE HERE
    time = np.array(time_step)

    # You should split the dataset into training and validation splits
    # At time 3000. So everything up to 3000 is training, and everything
    # after 3000 is validation. Write the code below to achieve that.
    split_time = 3000

    # YOUR CODE HERE
    time_train = time[:split_time]
    x_train = series[:split_time]
    time_valid = time[split_time:]
    x_valid = series[split_time:]
    # DO NOT CHANGE THIS CODE
    window_size = 30
    batch_size = 32
    shuffle_buffer_size = 1000


    tf.keras.backend.clear_session()
    # You can use any random seed you want. We use 53. :)
    tf.random.set_seed(53)
    np.random.seed(53)
    train_set = windowed_dataset(x_train, window_size=window_size, batch_size=batch_size, shuffle_buffer=shuffle_buffer_size)
    
    validation_set = windowed_dataset(x_valid, 
        window_size=window_size,
        batch_size=batch_size,
        shuffle_buffer=shuffle_buffer_size)


    model = Sequential([
        Conv1D(filters=64, kernel_size=5,
        padding="causal",
        activation="relu",
        input_shape=[None, 1],
        dtype=tf.float32),
        BatchNormalization(dtype=tf.float32),
        Dropout(0.5, dtype=tf.float32),
        LSTM(64, return_sequences=True, dtype=tf.float32),
        BatchNormalization(dtype=tf.float32),
        Dropout(0.5, dtype=tf.float32),
        Dense(128, activation="relu", dtype=tf.float32),
        LSTM(32, dtype=tf.float32),
        Dense(64, activation="relu", dtype=tf.float32),
        BatchNormalization(dtype=tf.float32),
        Dense(32, activation="relu", dtype=tf.float32),    
        Dense(1, dtype=tf.float32),
        Lambda(lambda x: x * 400, dtype=tf.float32)
    ])

   
    # YOUR CODE HERE TO COMPILE AND TRAIN THE MODEL
    optimizer = Adam(learning_rate=1e-4)
    loss= Huber()
    model.compile(loss=loss,
        optimizer=optimizer,
        metrics=["mae"])

    checkpoint_path = 'checkpoint.ckpt'
    checkpoint = ModelCheckpoint(checkpoint_path, 
        save_weights_only=True, 
        save_best_only=True, 
        monitor='val_mae',
        verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

    epochs=100

    history = model.fit(train_set, 
        validation_data=validation_set, 
        epochs=epochs, 
        callbacks=[checkpoint, early_stopping, reduce_lr],
        )

    model.load_weights(checkpoint_path)
    return model


# Note that you'll need to save your model as a .h5 like this
# This .h5 will be uploaded to the testing infrastructure
# and a score will be returned to you
if __name__ == '__main__':
    model = solution_model()
    model.save("tf5-sunspot-3.h5")

# 시험때 다음 줄은 복사하지 마세요
    # from google.colab import files
    # files.download('tf5-sunspot-3.h5')

Epoch 1/100
     93/Unknown - 7s 13ms/step - loss: 175.2579 - mae: 175.7573
Epoch 1: val_mae improved from inf to 67.85394, saving model to checkpoint.ckpt
93/93 [==============================] - 9s 29ms/step - loss: 175.2579 - mae: 175.7573 - val_loss: 67.3547 - val_mae: 67.8539 - lr: 1.0000e-04
Epoch 2/100
93/93 [==============================] - ETA: 0s - loss: 97.7120 - mae: 98.2109
Epoch 2: val_mae improved from 67.85394 to 66.92551, saving model to checkpoint.ckpt
93/93 [==============================] - 2s 24ms/step - loss: 97.7120 - mae: 98.2109 - val_loss: 66.4264 - val_mae: 66.9255 - lr: 1.0000e-04
Epoch 3/100
92/93 [============================>.] - ETA: 0s - loss: 74.2553 - mae: 74.7539
Epoch 3: val_mae improved from 66.92551 to 61.03418, saving model to checkpoint.ckpt
93/93 [==============================] - 2s 16ms/step - loss: 74.2230 - mae: 74.7216 - val_loss: 60.5361 - val_mae: 61.0342 - lr: 1.0000e-04
Epoch 4/100
89/93 [===========================>..] - ETA: 0s - lo

# 직접 코딩

과제 순서를 바꿔서, ChatGPT 가 짜 준 코드를 직접 개선해 본다

In [ ]:
# ======================================================================
# There are 5 questions in this test with increasing difficulty from 1-5
# Please note that the weight of the grade for the question is relative
# to its difficulty. So your Category 1 question will score much less
# than your Category 5 question.
# ======================================================================
#
# Sequence Modelling Question.
#
# For this task you will need to train a neural network
# to predict sunspot activity using the Sunspots.csv
# provided. Your neural network is expected to have an MAE
# of at least 20, with top marks going to one with an MAE
# of around 15. At the bottom is provided some testing
# code should you want to check before uploading which measures
# the MAE for you. Strongly recommend you test your model with
# this to be able to see how it performs.
import csv
import tensorflow as tf
import numpy as np
import urllib
from tensorflow.keras.layers import Dense, LSTM, Lambda, Conv1D,Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import Huber

# DO NOT CHANGE THIS CODE
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    # Window Sequence Loader 만들기
    series = tf.expand_dims(series, axis=-1)
    # Dataset loader 만들기
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)


def solution_model():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv'
    urllib.request.urlretrieve(url, 'sunspots.csv')

    # Your data should be loaded into 2 Python lists called time_step
    # and sunspots. They are decleared here.
    time_step = []
    sunspots = []

    with open('sunspots.csv') as csvfile:
      reader = csv.reader(csvfile, delimiter=',')
      next(reader)
      for row in reader:
        sunspots.append(float(row[2]))# YOUR CODE HERE
        time_step.append(int(row[0]))

    
    # You should use numpy to create 
    # - your series from the list of sunspots
    # - your time details from the list of time steps
    series = np.array(sunspots) # YOUR CODE HERE
    time = np.array(time_step)

    # You should split the dataset into training and validation splits
    # At time 3000. So everything up to 3000 is training, and everything
    # after 3000 is validation. Write the code below to achieve that.
    split_time = 3000

    # YOUR CODE HERE
    time_train = time[:split_time]
    x_train = series[:split_time]
    time_valid = time[split_time:]
    x_valid = series[split_time:]
    # DO NOT CHANGE THIS CODE
    window_size = 30
    batch_size = 32
    shuffle_buffer_size = 1000


    tf.keras.backend.clear_session()
    # You can use any random seed you want. We use 53. :)
    tf.random.set_seed(53)
    np.random.seed(53)
    train_set = windowed_dataset(x_train, window_size=window_size, batch_size=batch_size, shuffle_buffer=shuffle_buffer_size)
    
    validation_set = windowed_dataset(x_valid, 
        window_size=window_size,
        batch_size=batch_size,
        shuffle_buffer=shuffle_buffer_size)

    # 과도한 Batch Normalization 과 Dropdout 은 성능 저하의 원인이 되므로 제거한다 😇
    model = Sequential([
        Conv1D(filters=64, kernel_size=5,
        padding="causal",
        activation="relu",
        input_shape=[None, 1],
        dtype=tf.float32),
        BatchNormalization(dtype=tf.float32),
        Dropout(0.5, dtype=tf.float32),
        Bidirectional(LSTM(64, return_sequences=True, dtype=tf.float32)),
        BatchNormalization(dtype=tf.float32),
        Dropout(0.5, dtype=tf.float32),
        Dense(128, activation="relu", dtype=tf.float32),
        Bidirectional(LSTM(32, dtype=tf.float32)),
        Dense(64, activation="relu", dtype=tf.float32),
        BatchNormalization(dtype=tf.float32),
        Dense(32, activation="relu", dtype=tf.float32),    
        Dense(1, dtype=tf.float32),
        Lambda(lambda x: x * 400, dtype=tf.float32)
    ])
   
    # YOUR CODE HERE TO COMPILE AND TRAIN THE MODEL
    # optimizer 는 모범답안의 SGD 그대로.
    optimizer = SGD(learning_rate=1e-4, momentum=0.9)
    loss= Huber()
    model.compile(loss=loss,
        optimizer=optimizer,
        metrics=["mae"])

    checkpoint_path = 'checkpoint.ckpt'
    checkpoint = ModelCheckpoint(checkpoint_path, 
        save_weights_only=True, 
        save_best_only=True, 
        monitor='val_mae',
        verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)

    # lr 추가
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

    epochs=100

    history = model.fit(train_set, 
        validation_data=validation_set, 
        epochs=epochs, 
        callbacks=[checkpoint, early_stopping, reduce_lr],
        )

    model.load_weights(checkpoint_path)
    return model


# Note that you'll need to save your model as a .h5 like this
# This .h5 will be uploaded to the testing infrastructure
# and a score will be returned to you
if __name__ == '__main__':
    model = solution_model()
    model.save("tf5-sunspot-3.h5")

# 시험때 다음 줄은 복사하지 마세요
    # from google.colab import files
    # files.download('tf5-sunspot-3.h5')

Epoch 1/100
     90/Unknown - 9s 21ms/step - loss: 55.9400 - mae: 56.4374
Epoch 1: val_mae improved from inf to 25.10450, saving model to checkpoint.ckpt
93/93 [==============================] - 11s 39ms/step - loss: 55.2828 - mae: 55.7802 - val_loss: 24.6101 - val_mae: 25.1045 - lr: 1.0000e-04
Epoch 2/100
91/93 [============================>.] - ETA: 0s - loss: 33.8299 - mae: 34.3265
Epoch 2: val_mae did not improve from 25.10450
93/93 [==============================] - 2s 19ms/step - loss: 33.8883 - mae: 34.3850 - val_loss: 30.7728 - val_mae: 31.2696 - lr: 1.0000e-04
Epoch 3/100
93/93 [==============================] - ETA: 0s - loss: 31.1712 - mae: 31.6674
Epoch 3: val_mae improved from 25.10450 to 21.74508, saving model to checkpoint.ckpt
93/93 [==============================] - 2s 21ms/step - loss: 31.1712 - mae: 31.6674 - val_loss: 21.2504 - val_mae: 21.7451 - lr: 1.0000e-04
Epoch 4/100
92/93 [============================>.] - ETA: 0s - loss: 30.8952 - mae: 31.3913
Epoch 4: val_m